In [1]:
import fasttext
import pandas as pd
model = fasttext.load_model("./model/ICD10.bin")

#text preprocessing
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
import string

In [6]:
#text preprocessing
def prepare_text(df):
    df['prepare_term'] = df['term'].str.lower()
    df['tokens'] = df['prepare_term'].apply(nltk.word_tokenize)
    df['no_punct_text'] = df['prepare_term'].str.translate(str.maketrans('', '', string.punctuation))
    stop_words = set(stopwords.words('english'))
    df['filtered_text'] = df['no_punct_text'].apply(lambda x: " ".join(word for word in x.split() if word.lower() not in stop_words))
    stemmer = PorterStemmer()
    df['stemmed_text'] = df['filtered_text'].apply(lambda x: " ".join(stemmer.stem(word) for word in x.split()))
    lemmatizer = WordNetLemmatizer()
    df['lemmatized_text'] = df['stemmed_text'].apply(lambda x: " ".join(lemmatizer.lemmatize(word) for word in x.split()))
    new_df = df[['lemmatized_text']].copy()
    return new_df

ระบุ Path file ของ Validation set

In [3]:
filename = './data/dataset.tsv'

In [7]:
#Validation set
validation_tsv = pd.read_csv(filename,sep='\t')
prepared_val = prepare_text(validation_tsv)
prepared_val

,lemmatized_text
0,infect caus staphylococcu coagulas neg
1,infect due staphylococcu coagulas neg
2,fetal infect caus staphylococcu aureu
3,earli neonat infect caus staphylococcu aureu
4,bacteremia caus methicillin resist staphylococ...
...,...
16563,atherosclerosi aortoiliac bypass graft
16564,atherosclerosi aortoiliac bypass graft
16565,peripher nerv neurostimul devic situ
16566,peripher nerv neuropacemak situ


In [8]:
#Predict
predict = model.predict(prepared_val['lemmatized_text'].to_list())
result = pd.DataFrame({'term':validation_tsv['term'].to_list(),
                       'codePredict':[p[0].strip('__label__') for p in predict[0]]})
result

,term,codePredict
0,infection caused by Staphylococcus Coagulase n...,A49.0
1,infection due to Staphylococcus Coagulase nega...,A49.0
2,fetal infection caused by Staphylococcus aureus,A49.0
3,early neonatal infection caused by Staphylococ...,A49.0
4,bacteremia caused by Methicillin resistant Sta...,A49.0
...,...,...
16563,atherosclerosis of aortoiliac bypass graft,Z95.8
16564,atherosclerosis aortoiliac bypass graft,Z95.8
16565,peripheral nerve neurostimulator device in situ,Z95.8
16566,peripheral nerve neuropacemaker in situ,Z95.8
